## Задача: классификация твитов по тональности

У нас есть датасет из твитов, про каждый указано, как он эмоционально окрашен: положительно или отрицательно. Задача: предсказывать эмоциональную окраску.

Классификацию по тональности используют в рекомендательных системах, чтобы понять, понравилось ли людям кафе, кино, etc.

Скачиваем куски датасета ([источник](http://study.mokoron.com/)): [положительные](https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0), [отрицательные](https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv).

In [1]:
# если у вас линукс / мак / collab или ещё какая-то среда, в которой работает wget, можно так:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv

--2020-04-09 18:22:59--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.1, 2620:100:6026:1::a27d:4601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2020-04-09 18:22:59--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1c6e5f661da18c09c3640a4138.dl.dropboxusercontent.com/cd/0/inline/A1jWhHRMZc7bxejSAp_7N1z9MKw-RRoP6mbDuSohvhEdkzIuZ9IO7TJvq1looa7S1rgrzKizLlVvZ0WumTv8x3wUoBbFlY5fu579jg6ehvIwBg/file# [following]
--2020-04-09 18:23:00--  https://uc1c6e5f661da18c09c3640a4138.dl.dropboxusercontent.com/cd/0/inline/A1jWhHRMZc7bxejSAp_7N1z9MKw-RRoP6mbDuSohvhEdkzIuZ9IO7TJvq1looa7S1rgrzKizLlVvZ0WumTv8x3wUoBbFlY5fu579jg6ehvIwBg/file
Resolving uc1c6e

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [7]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)

negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)

df = positive.append(negative)

In [10]:
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [9]:
df.shape

(226834, 2)

In [11]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.label)

In [12]:
print(df.shape)
print(x_train.shape)
print(x_test.shape)

(226834, 2)
(170125,)
(56709,)


## Baseline: классификация необработанных n-грамм

### Векторизаторы

In [13]:
df['text'].head().tolist()

['@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)',
 'Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D',
 'RT @KatiaCheh: Ну ты идиотка) я испугалась за тебя!!!',
 'RT @digger2912: "Кто то в углу сидит и погибает от голода, а мы ещё 2 порции взяли, хотя уже и так жрать не хотим" :DD http://t.co/GqG6iuE2…',
 '@irina_dyshkant Вот что значит страшилка :D\nНо блин,посмотрев все части,у тебя создастся ощущение,что авторы курили что-то :D']

In [13]:
from sklearn.linear_model import LogisticRegression # можно заменить на любимый классификатор
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
vec = CountVectorizer()

In [15]:
texts = ['я живу в москве', 'в москве карантин', 'карантин карантин, я хочу гулять по москве']

In [16]:
vec.fit(texts)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [17]:
vec.vocabulary_

{'гулять': 0, 'живу': 1, 'карантин': 2, 'москве': 3, 'по': 4, 'хочу': 5}

In [21]:
toy_vectors = vec.transform(texts)

In [22]:
toy_vectors

<3x6 sparse matrix of type '<class 'numpy.int64'>'
	with 9 stored elements in Compressed Sparse Row format>

In [23]:
toy_vectors.todense()

matrix([[0, 1, 0, 1, 0, 0],
        [0, 0, 1, 1, 0, 0],
        [1, 0, 2, 1, 1, 1]])

Самый простой способ извлечь фичи из текстовых данных -- векторизаторы: `CountVectorizer` и `TfidfVectorizer`

Объект `CountVectorizer` делает простую вещь:
* строит для каждого документа (каждой пришедшей ему строки) вектор размерности `n`, где `n` -- количество слов или n-грам во всём корпусе
* заполняет каждый i-тый элемент количеством вхождений слова в данный документ

In [24]:
vec = CountVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train) # bow -- bag of words (мешок слов)

`ngram_range` отвечает за то, какие n-граммы мы используем в качестве фичей:<br/>
* ngram_range=(1, 1) -- униграммы<br/>
* ngram_range=(3, 3) -- триграммы<br/>
* ngram_range=(1, 3) -- униграммы, биграммы и триграммы.

В `vec.vocabulary_` лежит словарь: мэппинг слов к их индексам:

In [25]:
list(vec.vocabulary_.items())[:20]

[('rtvepyodds', 74653),
 ('адидасе', 98464),
 ('специфициский', 214979),
 ('pkolyan90', 69227),
 ('artemkrott', 14714),
 ('медведь', 157336),
 ('вдохновляют', 110769),
 ('otvgb1evdt', 67152),
 ('11декабря', 724),
 ('спасатель', 214763),
 ('любаш', 154485),
 ('боков', 107038),
 ('гангстер', 119025),
 ('dekkerst', 24669),
 ('getwellsoonluhan', 34100),
 ('боясь', 107930),
 ('javyjyzosuk', 41942),
 ('выбирешь', 116466),
 ('shennonforever', 77723),
 ('sunshine_olga', 81204)]

In [26]:
bow

<170125x243983 sparse matrix of type '<class 'numpy.int64'>'
	with 1846493 stored elements in Compressed Sparse Row format>

#### Обучаем и оцениваем

In [27]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)

/home/maryszmary/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [28]:
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.77      0.76      0.76     28374
    positive       0.76      0.77      0.77     28335

    accuracy                           0.77     56709
   macro avg       0.77      0.77      0.77     56709
weighted avg       0.77      0.77      0.77     56709



Попробуем сделать то же самое для триграмм:

In [29]:
vec = CountVectorizer(ngram_range=(3, 3))
bow = vec.fit_transform(x_train)

In [31]:
list(vec.vocabulary_.items())[:10]

[('успокоил но его', 1213349),
 ('дисциплин этой сессии', 380866),
 ('грустит жду завтра', 346628),
 ('chrissrichii polinakeat пользуясь', 33461),
 ('ходила искала фирму', 1235283),
 ('shimp93 да мне', 157459),
 ('тоже иду понедельник', 1160368),
 ('спит запеку горшочках', 1080628),
 ('администрация президента сейчас', 184928),
 ('власть попытка оказалась', 271022)]

In [33]:
bow.shape

(170125, 1327701)

In [32]:
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/home/maryszmary/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

    negative       0.47      0.72      0.57     18464
    positive       0.82      0.61      0.70     38245

    accuracy                           0.65     56709
   macro avg       0.64      0.66      0.63     56709
weighted avg       0.70      0.65      0.66     56709



(Как вы думаете, почему в результатах теперь такой разброс по сравнению с униграммами?)

## TF-IDF векторизация

`TfidfVectorizer` делает то же, что и `CountVectorizer`, но в качестве значений – tf-idf каждого слова.

Как считается tf-idf:

TF (term frequency) – относительная частотность слова в документе:
$$ TF(t,d) = \frac{n_t}{\sum_k n_k} $$

`t` -- слово (term), `d` -- документ, $n_t$ -- количество вхождений слова, $n_k$ -- количество вхождений остальных слов

IDF (inverse document frequency) – обратная частота документов, в которых есть это слово:
$$ IDF(t, D) = \mbox{log} \frac{|D|}{|{d : t \in d}|} $$

`t` -- слово (term), `D` -- коллекция документов

Перемножаем их:
$$TFIDF(t,d,D) = TF(t,d) \times IDF(i, D)$$

Сакральный смысл – если слово часто встречается в одном документе, но в целом по корпусу встречается в небольшом 
количестве документов, у него высокий TF-IDF.

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [35]:
tfidf = TfidfVectorizer()
toy_vectors = tfidf.fit_transform(texts)

In [37]:
toy_vectors.todense()

matrix([[0.        , 0.861037  , 0.        , 0.50854232, 0.        ,
         0.        ],
        [0.        , 0.        , 0.78980693, 0.61335554, 0.        ,
         0.        ],
        [0.42024133, 0.        , 0.63920872, 0.2482013 , 0.42024133,
         0.42024133]])

In [38]:
vec = TfidfVectorizer(ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/home/maryszmary/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

    negative       0.74      0.77      0.75     26858
    positive       0.78      0.75      0.77     29851

    accuracy                           0.76     56709
   macro avg       0.76      0.76      0.76     56709
weighted avg       0.76      0.76      0.76     56709



В этот раз получилось хуже :( 

## Убираем стоп-слова и пунктуацию

*Стоп-слова* -- это слова, которые часто встречаются практически в любом тексте и ничего интересного не говорят о конретном документе, то есть играют роль шума. Поэтому их принято убирать. По той же причине убирают и пунктуацию.

In [1]:
# import nltk
# nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/maryszmary/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [39]:
# у вас здесь, возможно, выскочит ошибка и надо будет загрузить стоп слова (как написано выше)
from nltk.corpus import stopwords
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [40]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [41]:
noise = stopwords.words('russian') + list(punctuation)

В векторизаторах за стоп-слова, логичным образом, отвечает аргумент `stop_words`.

In [42]:
from nltk import word_tokenize

Получилось чууть лучше. Что ещё можно сделать? Лемматизацию или стемминг.

## О важности эксплоративного анализа

Но иногда пунктуация бывает и не шумом -- главное отталкиваться от задачи. Что будет если вообще не убирать пунктуацию?

In [43]:
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=noise)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/home/maryszmary/.local/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['``'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/maryszmary/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

    negative       0.80      0.77      0.78     29379
    positive       0.76      0.80      0.78     27330

    accuracy                           0.78     56709
   macro avg       0.78      0.78      0.78     56709
weighted avg       0.78      0.78      0.78     56709



In [44]:
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/home/maryszmary/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     28138
    positive       1.00      1.00      1.00     28571

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



Стоило оставить пунктуацию -- и все метрики равны 1! Как это получилось? Среди неё были очень значимые токены (как вы думаете, какие?). Найдите фичи с самыми большими коэффициэнтами:

Посмотрим, как один из супер-значительных токенов справится с классификацией безо всякого машинного обучения:

In [45]:
cool_token = ')'
pred = ['positive' if cool_token in tweet else 'negative' for tweet in x_test]
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       1.00      0.85      0.92     33038
    positive       0.83      1.00      0.91     23671

    accuracy                           0.91     56709
   macro avg       0.91      0.93      0.91     56709
weighted avg       0.93      0.91      0.91     56709



## Символьные n-граммы

Теперь в качестве фичей используем, например, униграммы символов:

In [46]:
vec = CountVectorizer(analyzer='char', ngram_range=(1, 1))
bow = vec.fit_transform(texts)

In [48]:
bow.todense()

matrix([[3, 0, 0, 3, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
         1],
        [2, 0, 2, 2, 0, 1, 0, 1, 2, 0, 1, 2, 1, 0, 1, 1, 1, 0, 0, 0, 0,
         0],
        [6, 1, 4, 1, 1, 1, 0, 2, 3, 1, 1, 4, 3, 1, 2, 1, 3, 2, 1, 1, 1,
         2]])

In [49]:
vec = CountVectorizer(analyzer='char', ngram_range=(1, 1))
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/home/maryszmary/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

    negative       0.99      1.00      1.00     27980
    positive       1.00      0.99      1.00     28729

    accuracy                           1.00     56709
   macro avg       1.00      1.00      1.00     56709
weighted avg       1.00      1.00      1.00     56709



В общем-то, теперь уже понятно, почему на этих данных здесь 1. Так или инчае, на символах классифицировать тоже можно: для некторых задач (например, для определения языка) фичи-символьные n-граммы оказываются очень значимыми.

Ещё одна замечательная особенность фичей-символов: токенизация и лемматизация не нужна, можно использовать такой подход для языков, у которых нет готвых анализаторов.